In [1]:
from lightfm import LightFM
from lightfm.data import Dataset
import pandas as pd
import numpy as np

# Prepare dataset
df = pd.read_csv("temp.csv")
dataset = Dataset()
dataset.fit(df['user'], df['name'])

(interactions, weights) = dataset.build_interactions([(row['user'], row['name'], row['rating']) for _, row in df.iterrows()])

# Train model
model = LightFM(no_components=10, loss='warp')  # 'warp' good for implicit, use 'mse' or 'logistic' for explicit
model.fit(interactions, epochs=30, num_threads=4)

/opt/homebrew/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
from lightfm.evaluation import precision_at_k

print("Precision:", precision_at_k(model, interactions, k=5).mean())


Precision: 0.9200001


In [11]:
def recommend_items(model, dataset, df, user_id, N=5):
    n_users, n_items = interactions.shape
    user_index = dataset.mapping()[0][user_id]
    item_mapping = {v: k for k, v in dataset.mapping()[2].items()}

    scores = model.predict(user_index, np.arange(n_items))
    top_items = np.argsort(-scores)[:N]

    return {user_id: [item_mapping[i] for i in top_items]}

# Example usage:
recommend_items(model, dataset, df, user_id='user5')


{'user5': ['VEGA Smooth Curl 19 mm Barrel Hair Curler With Adjustable Temperature & Ceramic Coated Plates, (VHCH-03)',
  'BY4U Laptop Tabletop Stand for Desk, Portable Stand, Adjustable Height, Foldable, Non-Slip Aluminum MacBook Computer Stan...',
  "GoSriKi Women's Cotton Blend Kurta with Palazzo & Dupatta",
  'SHREGO Peanut Plus Roasted Peanut Salted 360G, Snack and Namkeen (2X180G Vacuum Packed)',
  'Apple Watch SE (GPS + Cellular, 44mm) - Space Grey Aluminium Case with Midnight Sport Band - Regular']}